In [62]:
import pandas as pd
import numpy as np

In [63]:
def make_categorical(df, col_name):
    df.loc[:, col_name] = pd.Categorical(df[col_name]) 

In [64]:
def mem_mib(df):
    print("{0:.2f} MiB".format(
        df.memory_usage().sum() / (1024 * 1024)
    ))

In [65]:
list_of_dataframes = []
df1 = pd.read_csv('raw-data/sensitivities_big4.csv', delimiter = ',', chunksize=100000)
instrumentTenorsAll = {}
for chunk in df1:
    columns_to_keep = ["ProformaShiftValue", "PartyName", "CLMS ID", "InstrumentTenor", "RiskMeasureType","UnderlyingTenor"]
    df_quick = chunk.filter(columns_to_keep)
    make_categorical(df_quick, "PartyName")
    make_categorical(df_quick, "InstrumentTenor")
    make_categorical(df_quick, "RiskMeasureType")
    make_categorical(df_quick, "UnderlyingTenor")
    instrumentTenors = pd.Categorical(df_quick["InstrumentTenor"])
    instrumentTenorsDict = dict(enumerate(instrumentTenors.categories))
    instrumentTenorsAll = {**instrumentTenorsAll, **instrumentTenorsDict}
    filtered = df_quick.loc[lambda x: (x.InstrumentTenor == '1M') | (x.InstrumentTenor == '2M')]
    list_of_dataframes.append(filtered)
    
df_final = pd.concat(list_of_dataframes)

In [40]:
instrumentTenorsAll

{0: '10Y',
 1: '12M',
 2: '15Y',
 3: '1D',
 4: '1M',
 5: '1W',
 6: '1Y',
 7: '20Y',
 8: '25Y',
 9: '2M',
 10: '2Y',
 11: '3M',
 12: '5Y',
 13: '6M'}

In [41]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 2 to 99
Data columns (total 6 columns):
ProformaShiftValue    19 non-null float64
PartyName             19 non-null category
CLMS ID               19 non-null int64
InstrumentTenor       19 non-null category
RiskMeasureType       19 non-null category
UnderlyingTenor       19 non-null category
dtypes: category(4), float64(1), int64(1)
memory usage: 7.9 KB


In [42]:
#df_final.info(memory_usage='deep')

In [43]:
df_final.shape[0]

19

In [44]:
exp = pd.DataFrame(instrumentTenorsAll.values())

In [45]:
instrumentTenorsAll.values()

dict_values(['10Y', '12M', '15Y', '1D', '1M', '1W', '1Y', '20Y', '25Y', '2M', '2Y', '3M', '5Y', '6M'])

In [60]:
query = "InstrumentTenor=='1M' & UnderlyingTenor=='5Y'"
dr = df_final.query(query)
dr.shape[0]

1

In [61]:
dr

,ProformaShiftValue,PartyName,CLMS ID,InstrumentTenor,RiskMeasureType,UnderlyingTenor
2,0.0025,Colin Young,17,1M,Interest Rate Delta,5Y
